In [ ]:
!pip install wandb -qU

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet
import wandb

In [ ]:
wandb.login()

wandb: Currently logged in as: boarkee. Use `wandb login --relogin` to force relogin


True

In [ ]:
wandb.init(project='Waste Sorting', entity='boarkee')

In [ ]:
IMG_SIZE = 224
NUM_CLASSES = 5

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory('/content/gdrive/MyDrive/image-classification/waste-sorting.v2i.folder/train',
                                                    target_size=(IMG_SIZE, IMG_SIZE),
                                                    batch_size=32,
                                                    class_mode='categorical')

Found 255 images belonging to 5 classes.


In [ ]:
validation_generator = test_datagen.flow_from_directory('/content/gdrive/MyDrive/image-classification/waste-sorting.v2i.folder/valid',
                                                        target_size=(IMG_SIZE, IMG_SIZE),
                                                        batch_size=32,
                                                        class_mode='categorical')

Found 68 images belonging to 5 classes.


In [ ]:
test_generator = test_datagen.flow_from_directory('/content/gdrive/MyDrive/image-classification/waste-sorting.v2i.folder/test',
                                                  target_size=(IMG_SIZE, IMG_SIZE),
                                                  batch_size=32,
                                                  class_mode='categorical')

Found 52 images belonging to 5 classes.


In [ ]:
base_model = MobileNet(input_shape=(IMG_SIZE, IMG_SIZE, 3),
                       include_top=False,
                       weights='imagenet')

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
predictions = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')(x)

In [ ]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator,
                    steps_per_epoch=len(train_generator),
                    epochs=10,
                    validation_data=validation_generator,
                    validation_steps=len(validation_generator))

Epoch 1/10
8/8 [==============================] - 25s 3s/step - loss: 0.6344 - accuracy: 0.7529 - val_loss: 0.0949 - val_accuracy: 0.9559
Epoch 2/10
8/8 [==============================] - 21s 2s/step - loss: 0.0960 - accuracy: 0.9647 - val_loss: 0.0640 - val_accuracy: 0.9853
Epoch 3/10
8/8 [==============================] - 19s 2s/step - loss: 0.0844 - accuracy: 0.9725 - val_loss: 0.0787 - val_accuracy: 0.9706
Epoch 4/10
8/8 [==============================] - 21s 3s/step - loss: 0.0206 - accuracy: 0.9922 - val_loss: 0.1084 - val_accuracy: 0.9706
Epoch 5/10
8/8 [==============================] - 21s 3s/step - loss: 0.0217 - accuracy: 0.9961 - val_loss: 0.0809 - val_accuracy: 0.9706
Epoch 6/10
8/8 [==============================] - 20s 3s/step - loss: 0.0089 - accuracy: 1.0000 - val_loss: 0.0528 - val_accuracy: 0.9706
Epoch 7/10
8/8 [==============================] - 21s 3s/step - loss: 0.0039 - accuracy: 1.0000 - val_loss: 0.0680 - val_accuracy: 0.9706
Epoch 8/10
8/8 [==================

In [ ]:
test_loss, test_acc = 0, 0 

In [ ]:
test_loss, test_acc = model.evaluate(test_generator, steps=len(test_generator))
print('Test accuracy:', test_acc)

2/2 [==============================] - 2s 793ms/step - loss: 0.1133 - accuracy: 0.9231
Test accuracy: 0.9230769276618958


In [ ]:
wandb.log({'test_loss': test_loss, 'test_acc': test_acc})

In [ ]:
wandb.finish()

test_acc,▁█
test_loss,▁█
test_acc,0.92308
test_loss,0.11333
